In [102]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import sqlite3
from tqdm.notebook import tqdm

##Генерация синтетических данных

#Таблицы: installs, sessions, events, onboarding.

In [103]:
np.random.seed(42)
random.seed(42)

In [104]:
N_USERS = 100000
START_DATE = datetime(2025, 11, 1)
END_DATE = datetime(2025, 12, 31)

In [105]:
users = []
user_id = 1
days = (END_DATE - START_DATE).days + 1

In [106]:
for day_offset in tqdm(range(days)):
    current_date = START_DATE + timedelta(days=day_offset)
    daily_installs = int(N_USERS / days * np.random.normal(1.0, 0.15))

    for _ in range(daily_installs):
        os = np.random.choice(['iOS', 'Android'], p=[0.45, 0.55])
        install_time = current_date + timedelta(
            hours=np.random.randint(0, 24),
            minutes=np.random.randint(0, 60)
        )

        users.append({
            'user_id': user_id,
            'install_date': install_time.date(),
            'install_timestamp': install_time,
            'os': os,
            'app_version': np.random.choice(['1.0.0', '1.0.1', '1.1.0'], p=[0.3, 0.5, 0.2]),
            'country': np.random.choice(['RU', 'US', 'EU'], p=[0.6, 0.25, 0.15])
        })
        user_id += 1


  0%|          | 0/61 [00:00<?, ?it/s]

In [107]:
df_installs = pd.DataFrame(users[:N_USERS])
df_installs.to_csv('installs.csv', index=False)

In [108]:
sessions = []
session_id = 1

In [109]:
for idx, user in tqdm(df_installs.iterrows()):
    user_id = user['user_id']
    install_date = user['install_date']
    os = user['os']

    if np.random.random() < 0.68:  # 68% R1
        first_session_date = install_date + timedelta(days=np.random.randint(0, 2))
        first_session_time = datetime.combine(
            first_session_date, datetime.min.time()
        ) + timedelta(
            hours=np.random.randint(0, 24),
            minutes=np.random.randint(0, 60)
        )
        session_duration = max(5, int(np.random.exponential(120)))

        sessions.append({
            'session_id': session_id,
            'user_id': user_id,
            'session_start': first_session_time,
            'session_end': first_session_time + timedelta(seconds=session_duration),
            'session_duration_sec': session_duration,
            'os': os,
            'event_type': 'session_open'
        })
        session_id += 1

df_sessions = pd.DataFrame(sessions)
df_sessions.to_csv('sessions.csv', index=False)

0it [00:00, ?it/s]

In [110]:
users = []
user_id = 1
days = (END_DATE - START_DATE).days + 1

In [111]:
for day_offset in tqdm(range(days)):
    current_date = START_DATE + timedelta(days=day_offset)
    daily_installs = int(N_USERS / days * np.random.normal(1.0, 0.15))

    for _ in range(daily_installs):
        os = np.random.choice(['iOS', 'Android'], p=[0.45, 0.55])
        install_time = current_date + timedelta(
            hours=np.random.randint(0, 24),
            minutes=np.random.randint(0, 60)
        )

        users.append({
            'user_id': user_id,
            'install_date': install_time.date(),
            'install_timestamp': install_time,
            'os': os,
            'app_version': np.random.choice(['1.0.0', '1.0.1', '1.1.0'], p=[0.3, 0.5, 0.2]),
            'country': np.random.choice(['RU', 'US', 'EU'], p=[0.6, 0.25, 0.15])
        })
        user_id += 1

df_installs = pd.DataFrame(users[:N_USERS])
df_installs.to_csv('installs.csv', index=False)

  0%|          | 0/61 [00:00<?, ?it/s]

In [112]:
events = []
event_id = 1

In [113]:
for idx, session in tqdm(df_sessions.iterrows()):
    user_id = session['user_id']
    session_id = session['session_id']
    session_start = session['session_start']

    if np.random.random() < 0.5:
        event_time = session_start + timedelta(seconds=np.random.randint(5, 30))

        events.append({
            'event_id': event_id,
            'user_id': user_id,
            'session_id': session_id,
            'event_type': 'workout_start',
            'event_timestamp': event_time,
            'os': session['os']
        })
        event_id += 1

        if np.random.random() < 0.68:
            workout_duration = np.random.randint(60, 600)
            event_time = event_time + timedelta(seconds=workout_duration)

            events.append({
                'event_id': event_id,
                'user_id': user_id,
                'session_id': session_id,
                'event_type': 'workout_complete',
                'event_timestamp': event_time,
                'os': session['os']
            })
            event_id += 1

df_events = pd.DataFrame(events)
df_events.to_csv('events.csv', index=False)

0it [00:00, ?it/s]

In [114]:
onboarding = []

In [115]:
for user_id in tqdm(df_installs['user_id']):
    if np.random.random() < 0.75:
        install_ts = pd.to_datetime(df_installs[df_installs['user_id'] == user_id]['install_timestamp'].values[0])
        start_time = install_ts + timedelta(seconds=np.random.randint(5, 60))

        completed = np.random.random() < 0.85

        if completed:
            end_time = start_time + timedelta(seconds=np.random.randint(30, 300))
        else:
            end_time = None

        onboarding.append({
            'user_id': user_id,
            'onboarding_started': start_time,
            'onboarding_completed': end_time,
            'completed': 1 if completed else 0
        })

df_onboarding = pd.DataFrame(onboarding)
df_onboarding.to_csv('onboarding.csv', index=False)

  0%|          | 0/97621 [00:00<?, ?it/s]

In [116]:
df_installs

,user_id,install_date,install_timestamp,os,app_version,country
0,1,2025-11-01,2025-11-01 00:31:00,iOS,1.1.0,US
1,2,2025-11-01,2025-11-01 07:39:00,Android,1.0.1,RU
2,3,2025-11-01,2025-11-01 00:30:00,iOS,1.0.1,RU
3,4,2025-11-01,2025-11-01 13:26:00,iOS,1.1.0,RU
4,5,2025-11-01,2025-11-01 02:58:00,Android,1.0.0,US
...,...,...,...,...,...,...
97616,97617,2025-12-31,2025-12-31 07:20:00,Android,1.0.0,RU
97617,97618,2025-12-31,2025-12-31 14:12:00,iOS,1.0.1,RU
97618,97619,2025-12-31,2025-12-31 05:29:00,iOS,1.1.0,RU
97619,97620,2025-12-31,2025-12-31 22:30:00,iOS,1.0.0,EU


In [117]:
df_sessions

,session_id,user_id,session_start,session_end,session_duration_sec,os,event_type
0,1,1,2025-11-02 13:21:00,2025-11-02 13:29:14,494,Android,session_open
1,2,3,2025-11-01 23:58:00,2025-11-02 00:01:08,188,iOS,session_open
2,3,5,2025-11-02 18:20:00,2025-11-02 18:22:39,159,iOS,session_open
3,4,8,2025-11-02 12:08:00,2025-11-02 12:11:47,227,Android,session_open
4,5,9,2025-11-02 06:33:00,2025-11-02 06:34:11,71,iOS,session_open
...,...,...,...,...,...,...,...
68164,68165,99994,2025-12-31 09:26:00,2025-12-31 09:27:51,111,iOS,session_open
68165,68166,99995,2025-12-31 20:46:00,2025-12-31 20:46:19,19,Android,session_open
68166,68167,99996,2026-01-01 21:46:00,2026-01-01 21:46:17,17,Android,session_open
68167,68168,99998,2026-01-01 01:01:00,2026-01-01 01:01:56,56,iOS,session_open


In [118]:
df_events

,event_id,user_id,session_id,event_type,event_timestamp,os
0,1,10,6,workout_start,2025-11-02 19:25:16,iOS
1,2,11,7,workout_start,2025-11-01 05:45:21,iOS
2,3,12,8,workout_start,2025-11-02 00:01:17,Android
3,4,12,8,workout_complete,2025-11-02 00:02:33,Android
4,5,14,10,workout_start,2025-11-02 19:19:12,Android
...,...,...,...,...,...,...
57290,57291,99994,68165,workout_start,2025-12-31 09:26:19,iOS
57291,57292,99995,68166,workout_start,2025-12-31 20:46:08,Android
57292,57293,99995,68166,workout_complete,2025-12-31 20:53:16,Android
57293,57294,99996,68167,workout_start,2026-01-01 21:46:29,Android


In [119]:
df_onboarding

,user_id,onboarding_started,onboarding_completed,completed
0,2,2025-11-01 07:39:27,2025-11-01 07:41:54,1
1,3,2025-11-01 00:30:21,2025-11-01 00:33:32,1
2,4,2025-11-01 13:26:42,2025-11-01 13:31:02,1
3,7,2025-11-01 08:15:40,2025-11-01 08:17:31,1
4,8,2025-11-01 12:43:56,2025-11-01 12:45:53,1
...,...,...,...,...
73153,97617,2025-12-31 07:20:36,2025-12-31 07:21:38,1
73154,97618,2025-12-31 14:12:22,NaT,0
73155,97619,2025-12-31 05:29:59,2025-12-31 05:34:02,1
73156,97620,2025-12-31 22:30:13,2025-12-31 22:34:35,1


##Загружаем датасеты в SQL

In [120]:
df_installs['install_date'] = pd.to_datetime(df_installs['install_date'])

In [121]:
df_sessions['session_start'] = pd.to_datetime(df_sessions['session_start'])

In [122]:
len(df_installs)

97621

In [123]:
conn = sqlite3.connect('analytics.db')

In [124]:
df_installs.to_sql('installs', conn, if_exists='replace', index=False)

97621

In [125]:
df_sessions.to_sql('sessions', conn, if_exists='replace', index=False)

68169

In [126]:
df_events.to_sql('events', conn, if_exists='replace', index=False)

57295

In [127]:
df_onboarding.to_sql('onboarding', conn, if_exists='replace', index=False)

73158

##Воронка конверсии

In [178]:
query = """
SELECT
    COUNT(DISTINCT i.user_id) as installs,
    COUNT(DISTINCT s.user_id) as reopens,
    COUNT(DISTINCT CASE WHEN e.event_type = 'workout_complete' THEN e.user_id END) as completions
FROM installs i
LEFT JOIN sessions s ON i.user_id = s.user_id
LEFT JOIN events e ON i.user_id = e.user_id
"""

In [182]:
pd.read_sql_query(query, conn)

,installs,reopens,completions
0,97621,66538,22672


In [183]:
result = pd.read_sql_query(query, conn)

In [184]:
result

,installs,reopens,completions
0,97621,66538,22672


In [131]:
total

np.int64(97621)

In [132]:
reopens

np.int64(66538)

In [133]:
completions

np.int64(22672)

##Retention

In [187]:
query = """
WITH user_first_return AS (
    SELECT
        i.user_id,
        i.install_date,
        MIN(CAST((julianday(s.session_start) - julianday(i.install_date)) AS INTEGER)) as days_to_return
    FROM installs i
    LEFT JOIN sessions s ON i.user_id = s.user_id
        AND s.session_start > datetime(i.install_date)
    GROUP BY i.user_id
)
SELECT
    COUNT(*) as total_users,
    SUM(CASE WHEN days_to_return = 1 THEN 1 ELSE 0 END) as d1_count,
    SUM(CASE WHEN days_to_return <= 7 THEN 1 ELSE 0 END) as d7_count,
    SUM(CASE WHEN days_to_return <= 30 THEN 1 ELSE 0 END) as d30_count
FROM user_first_return
"""

In [190]:
result = pd.read_sql_query(query, conn)

In [191]:
result

,total_users,d1_count,d7_count,d30_count
0,97621,2493,19480,19480


In [171]:
result.to_csv("ret1.csv", index=False)

from google.colab import files
files.download("ret.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##onboarding повышает шанс возврата

In [192]:
query = """
SELECT
    CASE WHEN o.completed = 1 THEN 'Completed' ELSE 'Not Completed' END as status,
    COUNT(*) as users,
    SUM(CASE WHEN s.user_id IS NOT NULL THEN 1 ELSE 0 END) as returned
FROM onboarding o
LEFT JOIN installs i ON o.user_id = i.user_id
LEFT JOIN sessions s ON o.user_id = s.user_id
    AND date(s.session_start) = date(i.install_date, '+1 day')
GROUP BY o.completed
"""


In [193]:
result = pd.read_sql_query(query, conn)

In [141]:
result

,status,users,returned
0,Not Completed,10786,283
1,Completed,62372,1593


In [142]:
(result['returned']/result['users'])*100

,0
0,2.623772
1,2.554031


##Сегментация по ОС

In [195]:
query = """
SELECT
    i.os,
    COUNT(DISTINCT i.user_id) as installs,
    COUNT(DISTINCT s.user_id) as users_with_session,
    ROUND(100.0 * COUNT(DISTINCT s.user_id) / COUNT(DISTINCT i.user_id), 1) as r1_pct,
    ROUND(AVG(s.session_duration_sec), 0) as avg_duration
FROM installs i
LEFT JOIN sessions s ON i.user_id = s.user_id
GROUP BY i.os
ORDER BY installs DESC
"""

In [196]:
result = pd.read_sql_query(query, conn)

In [145]:
result

,os,installs,users_with_session,r1_pct,avg_duration
0,Android,53326,36363,68.2,121.0
1,iOS,44295,30175,68.1,119.0


##DAU

In [198]:
query = """
SELECT
    date(session_start) as date,
    COUNT(DISTINCT user_id) as dau
FROM sessions
GROUP BY date(session_start)
ORDER BY date DESC
LIMIT 10
"""

In [199]:
result = pd.read_sql_query(query, conn)

In [148]:
result

,date,dau
0,2026-01-01,63
1,2025-12-31,748
2,2025-12-30,1240
3,2025-12-29,1135
4,2025-12-28,1104
5,2025-12-27,1146
6,2025-12-26,1055
7,2025-12-25,1087
8,2025-12-24,1119
9,2025-12-23,1257


In [149]:
result['dau'].mean()

np.float64(995.4)

##Сегментация по странам

In [150]:
query = """
SELECT
    country,
    COUNT(DISTINCT user_id) as users,
    ROUND(100.0 * COUNT(DISTINCT user_id) / (SELECT COUNT(*) FROM installs), 1) as pct
FROM installs
GROUP BY country
ORDER BY users DESC
"""

In [151]:
pd.read_sql_query(query, conn)

,country,users,pct
0,RU,58525,60.0
1,US,24321,24.9
2,EU,14775,15.1


##Когортный анализ

In [209]:
query = """
WITH user_first_return AS (
  SELECT
    i.user_id,
    date(i.install_date) AS install_date,
    MIN(
      CAST(julianday(date(s.session_start)) - julianday(date(i.install_date)) AS INTEGER)
    ) AS days
  FROM installs i
  LEFT JOIN sessions s
    ON i.user_id = s.user_id
   AND date(s.session_start) > date(i.install_date)   -- убираем day0
  GROUP BY i.user_id, date(i.install_date)
)
SELECT
  install_date,
  COUNT(*) AS cohort_size,
  SUM(CASE WHEN days = 1 THEN 1 ELSE 0 END) AS d1,
  ROUND(100.0 * SUM(CASE WHEN days = 1 THEN 1 ELSE 0 END) / COUNT(*), 1) AS d1_pct
FROM user_first_return
WHERE install_date <= date((SELECT MAX(date(session_start)) FROM sessions), '-1 day')
GROUP BY install_date;


"""

In [210]:
result=pd.read_sql_query(query, conn)

In [211]:
result[result['d1_pct']>0]

,install_date,cohort_size,d1,d1_pct
0,2025-11-01,1380,480,34.8
1,2025-11-02,1734,469,27.0
2,2025-11-03,1354,369,27.3
3,2025-11-04,1105,286,25.9
4,2025-11-05,1128,17,1.5
5,2025-11-06,1883,93,4.9
6,2025-11-07,1343,31,2.3
9,2025-11-10,1899,85,4.5
10,2025-11-11,1703,219,12.9
11,2025-11-12,1114,128,11.5
